# DriHRO prototype

This is a HRO prototype.

It should do:
    
    1. Get Ganttic projects
    2. Get CRM projects
    3. Reconcile both and launch warnings if anything is amiss.
    4. (Optional) Modify and edit stuff automatically
    5. (Optional) Get Factorial availability
    6. Generate a viz of available resources
    7. Generate a forecast of future income
    

In [1]:
from dotenv import load_dotenv
import os
import pandas as pd
import datetime

import driganttic.client as dg_client
import driganttic.parse as dg_parse

In [2]:
def to_df(ResourceList):
    """Parses resourcelist to dataframe"""
    return pd.DataFrame(data=ResourceList.dict()['fetched_items'])
def filter_tasks(tasks, field='end', date= datetime.datetime.today()):
    m = tasks.end > date
    return tasks.loc[m]

# Get Ganttic data

In [3]:
# API KEY is stored in the env file
load_dotenv()
APIKEY = os.getenv("APIKEY")

In [4]:
Client = dg_client.GantticClient(APIKEY=APIKEY)
# get all projects, tasks and resources
projects = Client.get_projects()
tasks = Client.get_tasks(timeMin = dg_parse.parse_timestamp('2021-01-01'),
                         timeMax= dg_parse.parse_timestamp('2022-04-30'))
resources = Client.get_resources()

In [5]:
df_pro = to_df(projects)
df_res = to_df(resources)
df_tas = to_df(tasks)

df_pro.set_index('id',drop=True, inplace=True)
df_tas.set_index('id',drop=True, inplace=True)
df_res.set_index('id',drop=True, inplace=True)

In [6]:
df_pro.head(1)

,fetched_timestamp,status,name,created,dateAproxStart,team,probability,service,scenario
id,,,,,,,,,
593543,2021-04-29 14:15:03.940461,project,Simon Dupond,2019-08-12 16:25:27,None,NaN,NaN,ServiceEnum.manteniment,ScenarioEnum.confirmat


In [7]:
df_res.head(1)

,fetched_timestamp,status,name,created,dedicacio,rol
id,,,,,,
795039,2021-04-29 14:15:03.940461,resource,Clotet,2019-08-12 16:26:08,100.0,RolEnum.soci


In [9]:
# Only active tasks today
df_tas = filter_tasks(df_tas)
df_tas['resources'] = df_tas.resources.map(lambda x: [df_res.loc[k]['name'] for k in x])
df_tas['project'] = df_tas.projectId.map(lambda x: df_pro.loc[x]['name'])
df_tas.head(1)

,fetched_timestamp,status,name,created,projectId,resources,start,end,utilizationPercent,project
id,,,,,,,,,,
18654630,2021-04-29 14:15:03.940461,task,DS,2019-08-12 16:47:35,593552,[Hoffmann],2021-01-01,2021-07-26,20.0,Marina CodiPress


In [36]:
# Only active projects
active_pros = df_tas.projectId.unique()
m = df_pro.index.isin(active_pros)
df_pro = df_pro.loc[m]
m2 = df_pro.scenario == 'Confirmat'
df_pro.loc[m2,'probability'] = 100
m3 = df_pro.dateAproxStart.isna()
df_pro.loc[m2 & m3, 'dateAproxStart'] = df_tas.groupby('projectId').start.min()
df_pro

,fetched_timestamp,status,name,created,dateAproxStart,team,probability,service,scenario
id,,,,,,,,,
593552,2021-04-29 14:15:03.940461,project,Marina CodiPress,2019-08-12 16:47:51,2020-07-17 05:15:00,3,100.0,ServiceEnum.manteniment,ScenarioEnum.confirmat
593557,2021-04-29 14:15:03.940461,project,Dribia Gestions,2019-08-12 16:57:53,2020-10-19 00:00:00,3,100.0,ServiceEnum.intern,ScenarioEnum.confirmat
629926,2021-04-29 14:15:03.940461,project,Dribia Comercial,2020-01-24 07:21:50,2019-12-02 00:00:00,2,100.0,ServiceEnum.intern,ScenarioEnum.confirmat
629930,2021-04-29 14:15:03.940461,project,CECOT Sofia,2020-01-24 07:32:12,2020-07-01 00:00:00,1,100.0,ServiceEnum.manteniment,ScenarioEnum.confirmat
640966,2021-04-29 14:15:03.940461,project,Simon Idea 2.0,2020-03-11 17:42:12,None,NaN,NaN,ServiceEnum.projecte,ScenarioEnum.optimista
670817,2021-04-29 14:15:03.940461,project,Inetum Gavius,2020-08-26 15:43:47,2021-07-29 00:00:00,3,100.0,ServiceEnum.projecte,ScenarioEnum.confirmat
679937,2021-04-29 14:15:03.940461,project,Simon GOIA fase 2-3,2020-10-14 12:31:33,2020-11-02 00:00:00,2,100.0,ServiceEnum.projecte,ScenarioEnum.confirmat
681151,2021-04-29 14:15:03.940461,project,Farmapremium year 2,2020-10-20 07:01:03,2021-04-06 00:00:00,2,100.0,ServiceEnum.projecte,ScenarioEnum.confirmat
681153,2021-04-29 14:15:03.940461,project,King year 6,2020-10-20 07:02:22,2021-02-01 00:00:00,3,100.0,ServiceEnum.projecte,ScenarioEnum.confirmat


## Perform checks

In [19]:
# perform checks
# No tasks with more than one person
assert((df_tas.resources.map(len) > 1).sum() == 0)
# No project without task
assert((df_tas.project.isna().sum() == 0))
# Total people
print('People role distribution')
print(df_res.rol.value_counts())
print('\n')
# Tasks per project
print('Tasks per project')
print(df_tas.project.value_counts())
# Types of project
print('\n')
print('Types of project')
print(df_pro.service.value_counts())
print('\n')
print('States of project')
print(df_pro.scenario.value_counts())

People role distribution
RolEnum.ds      11
RolEnum.lds      3
RolEnum.soci     3
RolEnum.bd       1
Name: rol, dtype: int64


Tasks per project
Adamo Laika              4
Adamo Olympus            4
Generalitat metadades    3
Adamo Pangea 3.0         3
Privalia Mèxic           3
Privalia Brasil          3
Dribia Gestions          3
King year 6              3
Inetum Gavius            3
Lucta year 2             3
Grífols Ichor            3
Marina CodiPress         3
Saplex Luigi             3
Simon GOIA fase 2-3      3
Danone YYY               2
Ventos PEED              2
Farmapremium year 2      2
Nedgia Canari            2
Dribia Comercial         2
CECOT Sofia              1
Dribia CRM+ PTQ          1
Simon Idea 2.0           1
Name: project, dtype: int64


Types of project
ServiceEnum.projecte       13
ServiceEnum.manteniment     5
ServiceEnum.intern          3
ServiceEnum.peed            1
Name: service, dtype: int64


States of project
ScenarioEnum.confirmat    20
ScenarioEnum.espe

## Warnings

In [37]:
# check empty teams, empty probabilities, empty startdates
print('Empty probabilities:', df_pro.probability.isna().sum())
print('Empty teams:', df_pro.team.isna().sum())
print('Empty startdates:', df_pro.dateAproxStart.isna().sum())

Empty probabilities: 1
Empty teams: 1
Empty startdates: 2
